# Standard Reservoir Computer (paper version)

### hyperparameters optimization

### ( datasets by $\rho \in A =$ list of values  )


In [1]:
# Import libraries
import math     as math
import numpy    as np
import networkx as nx
import random   as random
import pandas   as pd
import time     as time

#######################################################################
# E N V I R O N M E N T   S E T   U P
#######################################################################
#---------------------------------------------------------------------#
# To compute elapsed time
#---------------------------------------------------------------------#
start_time = time.time()

## variables description
    D       :(int  ) Input data dimension      
    N       :(int  ) Reservoir dimension (degrees of freedome of the reservoir)       
    rhoSR   :(float) Spectral Radius of A      
    rhoA    :(float) Density of A              
    alpha   :(float) Leak (or leakage) rate in (0,1]                 
    sigma   :(float) Strength of input signal            
    sigmab  :(float) Strength of input bias               
    beta    :(float) Tichonov-Miller regularization parameter
    
    washout :(int)   During training, skipped transitory timesteps in W_out calculation
    spinup  :(int)   Time (n. of timesteps) it takes for a trained RC to converge from its initial condition
                     onto the synchronization manifold to which it is driven by the input data
    normtime:(int)   Range to skip some QR factorisations and speed up the calculations  
            
    r       :(float) Reservoir state
    W_in    :(float) Input matrix              
    A       :(float) Reservoir ajacency matrix 
    b       :(float) bias vector               
    W_out   :(float) Output matrix  
    endtr_r :(float) Last reservoir state in training
    
    R       :(float) Matrix containing r(t) for all t in training dataset
    U       :(float) Matrix containing u(t) for all t in training dataset
    u       :(float) Time series at time t
    v       :(float) Prediction


In [2]:
# ReservoirComputer class declaration
class ReservoirComputer:
    def __init__(self, D, N, rhoSR, rhoA, alpha, sigma, sigmab, beta):
        self.r      = np.zeros(N)
        self.W_in   = get_random_matrix(N, D, xa=-sigma, xb=sigma, nonzero=False)
        self.A      = generate_adjacency_matrix(N, rhoSR, rhoA)
        self.b      = sigmab * np.ones(N)
        self.W_out  = np.zeros((D,N))
        self.endtr_r= np.zeros(N)

    def rc_update_rule(self, y):
        # driven     mode: y = u(t)         ;r(t+1) = F^d_r (r(t),y)
        # autonomous mode: y = W_out . r(t) ;r(t+1) = F^a_r (r(t),y)
        g      = np.dot(self.A, self.r) + np.dot(self.W_in, y) + self.b
        self.r = alpha * np.tanh(g) + (1 - alpha) * self.r

    def update_v(self):
        return np.dot(self.W_out, self.r)

    def train(self, U, washout):
        steps = U.shape[0]
        R     = np.zeros((N, steps))
        for i in range(steps):
            R[:, i] = self.r
            u       = U[i]
            self.rc_update_rule(u)
        self.W_out = linear_regression(R[:,washout:], U[washout:], beta)
        self.endtr_r = self.r # save last training r state

    def spinup(self, U, steps):
        self.r = self.endtr_r # reset reservoir state
        if steps > 0:
            for i in range(steps):
                u = U[i]
                self.rc_update_rule(u) # driven mode
    
    def predict(self, steps):
        prediction = np.zeros((steps, D))
        for i in range(steps):
            v             = self.update_v()
            prediction[i] = v
            self.rc_update_rule(v)
        return prediction

    def rc_lyapunov_exponents(self, steps, dt, normtime):
        save_r = self.r # save r state
        self.r = self.endtr_r # reset r state
        lyap   = np.zeros((N,steps))
        M      = np.eye(N)
        W      = self.A + np.dot(self.W_in,self.W_out)
        j      = -1
        for i in range(steps):
            v     = self.update_v()
            self.rc_update_rule(v) # update r
            #
            g     = np.dot(W, self.r) + self.b
            DF    = alpha * np.dot(np.diag(1 - np.tanh(g)**2),W) \
                    + (1 - alpha) * np.eye(N)
            Mn    = np.dot(DF,M)
            if (i % normtime == 0):
                Q,Rii = np.linalg.qr(Mn)
                j     += 1
                lyap[:,j] = np.log(abs(np.diag(Rii)))
                M = Q
        L = np.sum(lyap,1) / ((j+1)*dt)    
        self.r = save_r # retrieve saved r_state
        return L

    def rc_conditional_lyapunov_exponents(self, U, dt, normtime):
        save_r = self.r # save r state
        self.r = np.zeros(N) # reset r state
        steps  = U.shape[0]
        lyap   = np.zeros((N,steps))
        M      = np.eye(N)
        j      = -1
        for i in range(steps):
            u       = U[i]
            self.rc_update_rule(u) # update r
            #
            g  = np.dot(self.A, self.r) + np.dot(self.W_in, u) + self.b
            DF = alpha * np.dot(np.diag(1 - np.tanh(g)**2),self.A) \
                 + (1 - alpha) * np.eye(N)
            Mn = np.dot(DF,M)
            if (i % normtime == 0):
                Q,Rii = np.linalg.qr(Mn)
                j     += 1
                lyap[:,j] = np.log(abs(np.diag(Rii)))
                M = Q
        CL = np.sum(lyap,1) / ((j+1)*dt)
        self.r = save_r # retrieve saved r_state
        return CL
    
# Helper functions
def generate_adjacency_matrix(N, rhoSR, rhoA):
    # Erdos-Reyni network
    graph = nx.gnp_random_graph(N, rhoA)
    adj   = nx.to_numpy_array(graph)
    # Ensure random_array is of the same shape as the graph adjacency matrix
    random_array = get_random_matrix(N, N, xa =-0.5, xb=0.5, nonzero=True)
    # Multiply graph adjacency matrix with random values
    rescaled     = adj * random_array
    return scale_matrix(rescaled, rhoSR)

def get_random_matrix(nrow, ncol, xa, xb, nonzero):
    B = np.zeros((nrow,ncol))
    for i in range(nrow):
        for j in range(ncol):
            if nonzero:
                while B[i,j] == 0:
                    B[i,j] = xa + (xb - xa) * np.random.rand()
            else:
                B[i,j] = xa + (xb - xa) * np.random.rand()
    return B

def scale_matrix(A, rhoSR):
    eigenvalues = np.linalg.eigvals(A)    # compute eigenvlaues
    sr = np.max(np.absolute(eigenvalues)) # compute spectral radius of A
    if sr > 0:
        A = A * rhoSR / sr                # rescaling matrix if A non zero
    return A

def linear_regression(R, U, beta=.0001): 
    Rt = np.transpose(R)
    inverse_part = np.linalg.inv(np.dot(R, Rt) + beta * np.identity(R.shape[0]))
    return np.dot(np.dot(U.T, Rt), inverse_part)


In [3]:
#######################################################################
# O P T I M I Z A T I O N   L O S S   F U N C T I O N 
#######################################################################
# spinup_list = list of spinups values for forecasting 
def loss_macro(RC,test_data,spinup_list,forecast_steps):
    loss_value = 0.
    for i in range(len(spinup_list)):
        spinup_steps = spinup_list[i]
        RC.spinup(test_data, spinup_steps)
        #
        pred_data = RC.predict(forecast_steps)
        #
        for j in range(forecast_steps):
            loss_value += np.linalg.norm(test_data[spinup_steps+j] - pred_data[j])**2 \
                        * np.exp(- (j+1) / (forecast_steps) )
    return loss_value


In [4]:
#######################################################################
# H Y P E R P A R A M E T E R S   L I S T 
#######################################################################
N_list      = [400]
rhoSR_list  = [0.2, 0.5, 0.8]
rhoA_list   = [0.02]
alpha_list  = [0.4, 0.6, 0.8]
sigma_list  = [0.084]
sigmab_list = [1.1, 1.3, 1.5, 1.7]
beta_list   = [8.5e-8]

cart_prod = [(a,b,c,d,e,f,g)
             for a in N_list       
             for b in rhoSR_list   
             for c in rhoA_list    
             for d in alpha_list   
             for e in sigma_list   
             for f in sigmab_list  
             for g in beta_list   ]

len_vec = len(cart_prod)
vec_loss_macro = np.zeros(len_vec)

print("HP list length =",len_vec)

HP list length = 36


In [5]:
#######################################################################
# L I S T   O F   R H O   V A L U E S
#######################################################################
rho_list  = np.arange(0, 331, 1)

rho_list  = np.append(rho_list, np.array([13.926667, 23.9, 24.058, 470./19., 30.485]))
rho_list  = np.append(rho_list, np.array([99.524, 100.795]))
rho_list  = np.append(rho_list, np.array([148.4, 166.07, 214.364, 233.5]))

rho_list  = np.sort(rho_list)

len_rho_list = len(rho_list)

print(len_rho_list)

342


In [6]:
#######################################################################
# O P T I M I Z A T I O N   L O O P 
#######################################################################
rnd_seed = 167
nome_file = 'df_RC400_seed' + str(rnd_seed) + '_HP_parte02'
s_from = 60
s_to   = 120
# store parameters and RC hyperparameters
RC_HP   = np.zeros((s_to - s_from, 11)) #np.zeros((len_rho_list, 11))

cnt = -1
for s in range(s_from, s_to): #range(len_rho_list):
    rho_Lorenz  = rho_list[s]
    # read dataset
    str_rho_Lorenz = (f'{rho_Lorenz:07.3f}').replace(".", "_")
    print("===== ",f'{s:>4d}) : Lorenz rho = {rho_Lorenz:>7.3f}')
    X = np.loadtxt('dataset/Lorenz_Dataset_'+str(str_rho_Lorenz)+'.csv',delimiter=";")
    n_timesteps = len(X)
    #print(n_timesteps)
    
    # splitting in training and test dataset
    data_length          = len(X) 
    training_percentage  = .8
    training_data_length = round(training_percentage * data_length) 
    #print("data_length          =",data_length)
    #print("training_data_length =",training_data_length)
    training_data = np.array(X[:training_data_length])
    test_data     = np.array(X[training_data_length:])
    
    # Setting random seed for repeteability
    
    np.random.seed(rnd_seed)
    random.seed(rnd_seed)
    
    # Set random forecast spinup list
    M_forecasts = 100
    S_forecast_steps = 500
    max_spinup  = len(test_data) - S_forecast_steps
    spinup_list = np.zeros(M_forecasts, dtype=np.uint32)
    for i in range(M_forecasts):
        spinup_list[i] = round(max_spinup * np.random.rand())    
    
    for i in range(len_vec):
        # Setting random seed for repeteability
        np.random.seed(rnd_seed)
        random.seed(rnd_seed)
        # set hyperparameter from cartesian product
        D      = 3
        N      = cart_prod[i][0]
        rhoSR  = cart_prod[i][1]
        rhoA   = cart_prod[i][2]
        alpha  = cart_prod[i][3]  
        sigma  = cart_prod[i][4]
        sigmab = cart_prod[i][5] 
        beta   = cart_prod[i][6]
        #
        model   = ReservoirComputer(D, N, rhoSR, rhoA, alpha, sigma, sigmab, beta)
        # training
        washout = 100 # transitory skipped timesteps
        model.train(training_data, washout)
        # compute loss macro value
        vec_loss_macro[i] = loss_macro(model,test_data,spinup_list,S_forecast_steps)
        #print("i =",i, vec_loss_macro[i])
        
        del model
        
    idx_vec_loss_macro = np.argmin(vec_loss_macro, axis=-1)
    print("      ===== best hyperparameters")
    print("            N       ",cart_prod[idx_vec_loss_macro][0])
    print("            rhoSR   ",cart_prod[idx_vec_loss_macro][1])
    print("            rhoA    ",cart_prod[idx_vec_loss_macro][2])
    print("            alpha   ",cart_prod[idx_vec_loss_macro][3])
    print("            sigma   ",cart_prod[idx_vec_loss_macro][4])
    print("            sigmab  ",cart_prod[idx_vec_loss_macro][5])
    print("            beta    ",cart_prod[idx_vec_loss_macro][6])
    print("            loss    ",vec_loss_macro[idx_vec_loss_macro])

    # store best hyperparameters
    cnt += 1
    RC_HP[cnt][0]  = rho_Lorenz
    RC_HP[cnt][1]  = D
    RC_HP[cnt][2]  = washout
    RC_HP[cnt][3]  = cart_prod[idx_vec_loss_macro][0]
    RC_HP[cnt][4]  = cart_prod[idx_vec_loss_macro][1]
    RC_HP[cnt][5]  = cart_prod[idx_vec_loss_macro][2]
    RC_HP[cnt][6]  = cart_prod[idx_vec_loss_macro][3]
    RC_HP[cnt][7]  = cart_prod[idx_vec_loss_macro][4]
    RC_HP[cnt][8]  = cart_prod[idx_vec_loss_macro][5]
    RC_HP[cnt][9]  = cart_prod[idx_vec_loss_macro][6]
    RC_HP[cnt][10] = vec_loss_macro[idx_vec_loss_macro]
    

=====    60) : Lorenz rho =  55.000
      ===== best hyperparameters
            N        400
            rhoSR    0.2
            rhoA     0.02
            alpha    0.4
            sigma    0.084
            sigmab   1.5
            beta     8.5e-08
            loss     8398230.024272135
=====    61) : Lorenz rho =  56.000
      ===== best hyperparameters
            N        400
            rhoSR    0.8
            rhoA     0.02
            alpha    0.6
            sigma    0.084
            sigmab   1.7
            beta     8.5e-08
            loss     8686624.677524483
=====    62) : Lorenz rho =  57.000
      ===== best hyperparameters
            N        400
            rhoSR    0.2
            rhoA     0.02
            alpha    0.6
            sigma    0.084
            sigmab   1.3
            beta     8.5e-08
            loss     4673242.41367163
=====    63) : Lorenz rho =  58.000
      ===== best hyperparameters
            N        400
            rhoSR    0.5
            

      ===== best hyperparameters
            N        400
            rhoSR    0.2
            rhoA     0.02
            alpha    0.4
            sigma    0.084
            sigmab   1.7
            beta     8.5e-08
            loss     9301933.687618513
=====    90) : Lorenz rho =  85.000
      ===== best hyperparameters
            N        400
            rhoSR    0.2
            rhoA     0.02
            alpha    0.4
            sigma    0.084
            sigmab   1.7
            beta     8.5e-08
            loss     15229740.303764634
=====    91) : Lorenz rho =  86.000
      ===== best hyperparameters
            N        400
            rhoSR    0.2
            rhoA     0.02
            alpha    0.4
            sigma    0.084
            sigmab   1.7
            beta     8.5e-08
            loss     14212808.8515496
=====    92) : Lorenz rho =  87.000
      ===== best hyperparameters
            N        400
            rhoSR    0.2
            rhoA     0.02
            alpha    

      ===== best hyperparameters
            N        400
            rhoSR    0.2
            rhoA     0.02
            alpha    0.4
            sigma    0.084
            sigmab   1.3
            beta     8.5e-08
            loss     32231039.426235918
=====   119) : Lorenz rho = 112.000
      ===== best hyperparameters
            N        400
            rhoSR    0.2
            rhoA     0.02
            alpha    0.4
            sigma    0.084
            sigmab   1.3
            beta     8.5e-08
            loss     35579105.296191595


In [7]:
# Convert numpy array to pandas DataFrame
df_RC_HP = pd.DataFrame(RC_HP)

# Name columns
df_RC_HP.columns =['rho_Lorenz',
                   'D',
                   'washout',
                   'N',        
                   'rhoSR',   
                   'rhoA',    
                   'alpha',    
                   'sigma',    
                   'sigmab',   
                   'beta',
                   'loss']

# Save DataFrame to .csv
df_RC_HP.to_csv('climate/'+ nome_file + '.csv', index=False, header=True, decimal='.', sep=';')

In [8]:
#---------------------------------------------------------------------#---------------------------------------------------------------------#
# Elapsed time
#---------------------------------------------------------------------#
print(f'\nElapsed time {time.time() - start_time:6.2e} s')


Elapsed time 3.10e+04 s
